# Example KubeHound queries 

This file contains examples queries for kubehound. This file should be mostly read only. No modification will be ported to the original file, it will be kept only for the duration of the docker container.

Note: You may need to adjust the value on the cell below and execute (ctrl + enter) before running any other cells.

## Initial Setup

Connect to the kubegraph server by running the cell below

In [ ]:
%%graph_notebook_config
{
  "host": "host.docker.internal",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  }
}

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

## Critical Path Queries

### All critical paths from external services

In [ ]:
%%gremlin -d class -g critical  -le 50 -p inv,oute
kh.services().criticalPaths().by(elementMap()).limit(100)

### All critical paths from any endpoint

In [ ]:
%%gremlin -d class -g critical  -le 50 -p inv,oute
kh.endpoints().criticalPaths().by(elementMap()).limit(100)

### All critical paths from groups

In [ ]:
%%gremlin -d class -g critical  -le 50 -p inv,oute
kh.groups().criticalPaths().by(elementMap()).limit(100)

### Complex critical path query example

In [ ]:
%%gremlin -d class -g critical  -le 50 -p inv,oute
kh.services().criticalPaths().by(elementMap()).limit(100)
TODO TODO

See the [documentation](https://kubehound.io/queries/dsl/#criticalpaths-step) for further examples of critical path analysis

## Security Posture Queries

In [ ]:
TODO min hops, critical path freq